# Genomics ML Notebook

This notebook walks through data exploration, gradient derivations, training visualizations, model comparisons, and convergence analysis for the custom linear and logistic regression implementations.

## 1. Import Libraries and Load Dataset

We load standard analysis libraries plus the project utilities. The data is loaded from `data/processed/cleaned_variants.csv` if available; otherwise we preprocess the raw file.

In [ ]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    mean_squared_error,
    precision_score,
    r2_score,
    recall_score,
    roc_auc_score,
)
from sklearn.linear_model import LinearRegression, LogisticRegression

HAS_PLOTLY = False
try:
    import plotly.graph_objects as go
    import plotly.express as px

    HAS_PLOTLY = True
except Exception:
    HAS_PLOTLY = False

root = Path("..").resolve()
if str(root) not in sys.path:
    sys.path.insert(0, str(root))

from src.preprocess_genomics import (
    DEFAULT_INPUT as RAW_DEFAULT,
    DEFAULT_OUTPUT as PROCESSED_DEFAULT,
    detect_format,
    normalize_dataframe,
    parse_csv,
    parse_vcf,
)
from src.linear_regression_scratch import LinearRegressionScratch
from src.logistic_regression_scratch import LogisticRegressionScratch

In [ ]:
raw_path = root / RAW_DEFAULT
processed_path = root / PROCESSED_DEFAULT

if processed_path.exists():
    df = pd.read_csv(processed_path)
else:
    if not raw_path.exists():
        fallback = root / "data/raw/synthetic_variants.csv"
        if fallback.exists():
            raw_path = fallback
        else:
            raise FileNotFoundError(
                "No raw data found. Run src/download_genomics_data.py first."
            )

    input_format = detect_format(str(raw_path))
    if input_format == "vcf":
        raw_df = parse_vcf(str(raw_path))
    else:
        raw_df = parse_csv(str(raw_path))

    df = normalize_dataframe(raw_df)
    processed_path.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(processed_path, index=False)

print(df.head())
print("Rows:", len(df), "Columns:", df.columns.tolist())

## 2. Explore Data and Visualize Distributions

We inspect summary statistics, missing values, and a few visualizations to understand distributions and relationships among numeric features.

In [ ]:
display(df.describe(include="all"))
missing = df.isna().sum().sort_values(ascending=False)
print(missing[missing > 0])

In [ ]:
numeric_df = df.select_dtypes(include=[np.number])

plt.figure(figsize=(10, 6))
numeric_df.hist(bins=30, figsize=(10, 6))
plt.tight_layout()
plt.show()

if "depth" in df.columns and "position" in df.columns:
    plt.figure(figsize=(6, 4))
    plt.scatter(df["position"], df["depth"], s=10, alpha=0.4)
    plt.xlabel("Position")
    plt.ylabel("Depth")
    plt.title("Depth vs Position")
    plt.tight_layout()
    plt.show()

corr = numeric_df.corr()
plt.figure(figsize=(6, 5))
plt.imshow(corr, cmap="viridis")
plt.colorbar(label="Correlation")
plt.xticks(range(len(corr.columns)), corr.columns, rotation=45, ha="right")
plt.yticks(range(len(corr.columns)), corr.columns)
plt.title("Correlation Heatmap")
plt.tight_layout()
plt.show()

if HAS_PLOTLY and "depth" in df.columns:
    fig = px.histogram(df, x="depth", title="Depth Distribution (Interactive)")
    fig.show()

## 3. Derive Gradients Step-by-Step

We derive gradients for both models.

**Linear Regression**

Model: $\hat{y} = Xw + b$

Loss (MSE):
$$
L = \frac{1}{n} \sum_{i=1}^n (\hat{y}_i - y_i)^2
$$

Gradients:
$$
\frac{\partial L}{\partial w} = \frac{2}{n} X^T(\hat{y} - y), \quad
\frac{\partial L}{\partial b} = \frac{2}{n} \sum_{i=1}^n (\hat{y}_i - y_i)
$$

**Logistic Regression**

Model: $p = \sigma(z)$ where $z = Xw + b$ and $\sigma(z) = 1/(1+e^{-z})$

Loss (BCE):
$$
L = -\frac{1}{n} \sum_{i=1}^n \left[y_i \log p_i + (1-y_i) \log(1-p_i)\right]
$$

Gradients:
$$
\frac{\partial L}{\partial w} = \frac{1}{n} X^T(p - y), \quad
\frac{\partial L}{\partial b} = \frac{1}{n} \sum_{i=1}^n (p_i - y_i)
$$

We verify these gradients numerically below.

In [ ]:
def finite_diff_grad(loss_fn, w, eps=1e-5):
    grad = np.zeros_like(w)
    for i in range(len(w)):
        w_pos = w.copy()
        w_neg = w.copy()
        w_pos[i] += eps
        w_neg[i] -= eps
        grad[i] = (loss_fn(w_pos) - loss_fn(w_neg)) / (2 * eps)
    return grad

# Linear regression gradient check
np.random.seed(42)
X = np.random.randn(20, 3)
y = np.random.randn(20)
w = np.random.randn(3)


def linear_loss(w_vec):
    preds = X @ w_vec
    return np.mean((preds - y) ** 2)

analytic_grad = (2.0 / len(X)) * (X.T @ (X @ w - y))
fd_grad = finite_diff_grad(linear_loss, w)
print("Linear grad diff (L2):", np.linalg.norm(analytic_grad - fd_grad))

# Logistic regression gradient check
X_log = np.random.randn(30, 4)
y_log = (np.random.rand(30) > 0.5).astype(float)
w_log = np.random.randn(4)


def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))


def log_loss_fn(w_vec):
    p = sigmoid(X_log @ w_vec)
    eps = 1e-12
    p = np.clip(p, eps, 1.0 - eps)
    return -np.mean(y_log * np.log(p) + (1 - y_log) * np.log(1 - p))

analytic_grad_log = (1.0 / len(X_log)) * (X_log.T @ (sigmoid(X_log @ w_log) - y_log))
fd_grad_log = finite_diff_grad(log_loss_fn, w_log)
print("Logistic grad diff (L2):", np.linalg.norm(analytic_grad_log - fd_grad_log))

## 4. Implement Training Loop and Log Metrics

We train the custom models and collect loss/accuracy histories for later visualization.

In [ ]:
numeric_df = df.select_dtypes(include=[np.number])

# Linear regression target
if "depth" in df.columns:
    X_lin = df.drop(columns=["depth", "label"], errors="ignore").select_dtypes(include=[np.number])
    y_lin = df["depth"]

    Xl_train, Xl_test, yl_train, yl_test = train_test_split(
        X_lin, y_lin, test_size=0.2, random_state=42
    )

    lin_model = LinearRegressionScratch(lr=0.01, n_iters=1000)
    lin_model.fit(Xl_train.values, yl_train.values)
else:
    lin_model = None

# Logistic regression target
if "label" in df.columns:
    X_log = df.drop(columns=["label"], errors="ignore").select_dtypes(include=[np.number])
    y_log = df["label"]

    Xg_train, Xg_test, yg_train, yg_test = train_test_split(
        X_log, y_log, test_size=0.2, random_state=42, stratify=y_log
    )

    log_model = LogisticRegressionScratch(lr=0.1, n_iters=1000)
    log_model.fit(Xg_train.values, yg_train.values)
else:
    log_model = None

## 5. Visualize Training Process

We plot loss curves (and accuracy for logistic regression) to understand training dynamics.

In [ ]:
fig_dir = root / "results/figures"
fig_dir.mkdir(parents=True, exist_ok=True)

if lin_model is not None:
    plt.figure(figsize=(6, 4))
    plt.plot(lin_model.loss_history)
    plt.xlabel("Iteration")
    plt.ylabel("MSE Loss")
    plt.title("Linear Regression Loss")
    plt.tight_layout()
    plt.savefig(fig_dir / "linear_loss_curve.png", dpi=150)
    plt.show()

if log_model is not None:
    plt.figure(figsize=(6, 4))
    plt.plot(log_model.loss_history, label="loss")
    plt.plot(log_model.acc_history, label="accuracy")
    plt.xlabel("Iteration")
    plt.title("Logistic Regression Loss/Accuracy")
    plt.legend()
    plt.tight_layout()
    plt.savefig(fig_dir / "logistic_training_curves.png", dpi=150)
    plt.show()

## 6. Compare Models and Report Results

We compare custom and sklearn implementations using standard metrics and coefficient summaries.

In [ ]:
results_dir = root / "results"
results_dir.mkdir(parents=True, exist_ok=True)

if lin_model is not None:
    sk_lin = LinearRegression().fit(Xl_train, yl_train)
    custom_pred = lin_model.predict(Xl_test.values)
    sk_pred = sk_lin.predict(Xl_test)

    lin_metrics = pd.DataFrame(
        [
            {
                "model": "custom",
                "mse": mean_squared_error(yl_test, custom_pred),
                "r2": r2_score(yl_test, custom_pred),
            },
            {
                "model": "sklearn",
                "mse": mean_squared_error(yl_test, sk_pred),
                "r2": r2_score(yl_test, sk_pred),
            },
        ]
    )
    display(lin_metrics)
    lin_metrics.to_csv(results_dir / "linear_metrics.csv", index=False)

    coef_table = pd.DataFrame(
        {
            "feature": Xl_train.columns,
            "custom_coef": lin_model.weights,
            "sklearn_coef": sk_lin.coef_,
        }
    )
    display(coef_table.head())
    coef_table.to_csv(results_dir / "linear_coefficients.csv", index=False)

if log_model is not None:
    sk_log = LogisticRegression(max_iter=2000).fit(Xg_train, yg_train)
    custom_proba = log_model.predict_proba(Xg_test.values)
    sk_proba = sk_log.predict_proba(Xg_test)[:, 1]

    custom_pred = (custom_proba >= 0.5).astype(int)
    sk_pred = (sk_proba >= 0.5).astype(int)

    log_metrics = pd.DataFrame(
        [
            {
                "model": "custom",
                "accuracy": accuracy_score(yg_test, custom_pred),
                "precision": precision_score(yg_test, custom_pred, zero_division=0),
                "recall": recall_score(yg_test, custom_pred, zero_division=0),
                "f1": f1_score(yg_test, custom_pred, zero_division=0),
                "roc_auc": roc_auc_score(yg_test, custom_proba),
            },
            {
                "model": "sklearn",
                "accuracy": accuracy_score(yg_test, sk_pred),
                "precision": precision_score(yg_test, sk_pred, zero_division=0),
                "recall": recall_score(yg_test, sk_pred, zero_division=0),
                "f1": f1_score(yg_test, sk_pred, zero_division=0),
                "roc_auc": roc_auc_score(yg_test, sk_proba),
            },
        ]
    )
    display(log_metrics)
    log_metrics.to_csv(results_dir / "logistic_metrics.csv", index=False)

    log_coef = pd.DataFrame(
        {
            "feature": Xg_train.columns,
            "custom_coef": log_model.weights,
            "sklearn_coef": sk_log.coef_.ravel(),
        }
    )
    display(log_coef.head())
    log_coef.to_csv(results_dir / "logistic_coefficients.csv", index=False)

## 7. Convergence Analysis with Interactive Plots

We compare loss curves for multiple learning rates and visualize coefficient evolution. If Plotly is available, plots are interactive.

In [ ]:
learning_rates = [0.001, 0.01, 0.1]

if log_model is not None:
    loss_map = {}
    grad_map = {}
    coef_map = {}

    for lr in learning_rates:
        model = LogisticRegressionScratch(lr=lr, n_iters=300)
        model.fit(Xg_train.values, yg_train.values)
        loss_map[lr] = model.loss_history
        grad_map[lr] = model.grad_history
        coef_map[lr] = np.vstack(model.coef_history)

    if HAS_PLOTLY:
        fig = go.Figure()
        for lr, losses in loss_map.items():
            fig.add_trace(go.Scatter(y=losses, mode="lines", name=f"lr={lr}"))
        fig.update_layout(title="Logistic Loss Curves", xaxis_title="Iteration", yaxis_title="Loss")
        fig.show()

        fig = go.Figure()
        for lr, grads in grad_map.items():
            fig.add_trace(go.Scatter(y=grads, mode="lines", name=f"lr={lr}"))
        fig.update_layout(title="Gradient Magnitudes", xaxis_title="Iteration", yaxis_title="L2 Norm")
        fig.show()
    else:
        for lr, losses in loss_map.items():
            plt.plot(losses, label=f"lr={lr}")
        plt.title("Logistic Loss Curves")
        plt.xlabel("Iteration")
        plt.ylabel("Loss")
        plt.legend()
        plt.tight_layout()
        plt.show()

    coef_lr = 0.01
    if coef_lr in coef_map:
        coef_hist = coef_map[coef_lr]
        max_features = min(6, coef_hist.shape[1])
        plt.figure(figsize=(7, 4))
        for i in range(max_features):
            plt.plot(coef_hist[:, i], label=Xg_train.columns[i])
        plt.title("Coefficient Evolution (lr=0.01)")
        plt.xlabel("Iteration")
        plt.ylabel("Coefficient")
        plt.legend(ncol=2, fontsize=8)
        plt.tight_layout()
        plt.savefig(fig_dir / "logistic_coef_evolution.png", dpi=150)
        plt.show()

## Conclusions and Insights

- Feature scaling and label balance strongly influence convergence for logistic regression.
- Learning rate $0.01$ is a stable default here; $0.1$ may converge faster but can oscillate.
- Custom implementations match sklearn trends when inputs are numeric and normalized.
- Coefficient evolution helps diagnose overfitting and unstable updates.
- Use the comparison tables to validate correctness before experimenting with new features.